In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM, Bidirectional, TimeDistributed, Dropout, Layer
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the datasets
train_df = pd.read_csv('/Users/arunaa/Python/Sracasam/Malayalam/sarcasm_mal_train.csv')
test_df = pd.read_csv('/Users/arunaa/Python/Sracasam/Malayalam/sarcasm_mal_test_without_labels.csv')

In [3]:
# Separate features and labels
X_train = train_df['Text']
y_train = train_df['labels']

In [4]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [5]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=100)

In [6]:
# Split the data for evaluation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_padded, y_train_encoded, test_size=0.2, random_state=42)

In [7]:
# Define the Attention layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], input_shape[-1]), initializer='uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(input_shape[-1],), initializer='uniform', trainable=True)
        self.u = self.add_weight(name='att_var', shape=(input_shape[-1],), initializer='uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        u_it = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.exp(K.sum(K.dot(u_it, K.expand_dims(self.u, -1)), axis=-1))
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        return K.sum(weighted_input, axis=1)

In [8]:
# Build the HAN model
input_layer = Input(shape=(100,))
embedding_layer = Embedding(input_dim=5000, output_dim=128, input_length=100)(input_layer)
lstm_layer = Bidirectional(LSTM(units=128, return_sequences=True))(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)
dropout_layer = Dropout(0.5)(attention_layer)
output_layer = Dense(units=1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train_split, y_train_split, epochs=10, batch_size=32, validation_data=(X_val_split, y_val_split), callbacks=[early_stopping])

Epoch 1/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 32s 94ms/step - accuracy: 0.8165 - loss: 0.4643 - val_accuracy: 0.8480 - val_loss: 0.3713
Epoch 2/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 36s 108ms/step - accuracy: 0.8859 - loss: 0.2912 - val_accuracy: 0.8525 - val_loss: 0.3809
Epoch 3/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 36s 110ms/step - accuracy: 0.9144 - loss: 0.2241 - val_accuracy: 0.8385 - val_loss: 0.4466
Epoch 4/10
330/330 ━━━━━━━━━━━━━━━━━━━━ 37s 111ms/step - accuracy: 0.9479 - loss: 0.1582 - val_accuracy: 0.8196 - val_loss: 0.4971


In [10]:
# Vectorize the test data
X_test = test_df['Text']
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=100)

In [11]:
# Make predictions on the test set
test_predictions_prob = model.predict(X_test_padded)
test_predictions = (test_predictions_prob > 0.5).astype(int)
test_predictions_labels = label_encoder.inverse_transform(test_predictions.flatten())

89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step


In [12]:
# Save predictions to CSV file
test_df['Predicted_Labels'] = test_predictions_labels
output_path = '/Users/arunaa/Python/Sracasam/Malayalam/predictions_HAN_mal.csv'
test_df.to_csv(output_path, index=False)

In [13]:
# Predict on the validation set
val_predictions_prob = model.predict(X_val_split)
val_predictions = (val_predictions_prob > 0.5).astype(int)
val_predictions_labels = label_encoder.inverse_transform(val_predictions.flatten())

83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step


In [14]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_val_split, val_predictions)
precision, recall, f1, support = precision_recall_fscore_support(y_val_split, val_predictions, average='weighted')
classification_rep = classification_report(y_val_split, val_predictions)

In [15]:
# Print the evaluation metrics and classification report
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.8479909021986354
F1 Score: 0.8259634530104297
Precision: 0.8319381929950196
Recall: 0.8479909021986354

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.97      0.91      2142
           1       0.70      0.34      0.46       496

    accuracy                           0.85      2638
   macro avg       0.78      0.65      0.68      2638
weighted avg       0.83      0.85      0.83      2638



In [16]:
# Print the dataset with text and predicted labels
print(test_df[['Text', 'Predicted_Labels']])

                                                   Text Predicted_Labels
0     Shavakallarayile Kuzhimaadathile Peril Oru Let...    Non-sarcastic
1     ഗീതു മോഹൻദാസ് മലയാള സിനിമക്കു നൽകുന്ന വമ്പൻ ഗി...    Non-sarcastic
2                      Ente ponno ah sound🥰🥰 poli poli🤘    Non-sarcastic
3     Villain sharafudheen  ennu thonnunnavar likikk...    Non-sarcastic
4                    pulimurukan trailer ano kanunath 🤔    Non-sarcastic
...                                                 ...              ...
2821  Ente ponno oru adaaru jagapoka aanenu manasila...    Non-sarcastic
2822  എന്റെ ഇക്ക nja നമിച്ചു... ഒരു രക്ഷയില്ല ഹെവി ഐ...    Non-sarcastic
2823  ദേ ഇപ്പൊ കണ്ട് ഇറങ്ങിയതേ ഉള്ളു  96  Karikku (+...    Non-sarcastic
2824  1) Drisyam 2) Memories  3) Seconds 4) Grand ma...    Non-sarcastic
2825  Super mammoookkkaaa... ....   Oru lalettan bha...    Non-sarcastic

[2826 rows x 2 columns]
